In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.3'
# spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 https://cloud.r-project.org/bin/linux/ubuntu bi

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-11-18 15:36:01--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.11MB/s    in 0.2s    

2021-11-18 15:36:02 (5.11 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [5]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("DataFrameBasics").getOrCreate()

In [8]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_DVD_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Video_DVD_v1_00.tsv.gz"), sep="\t", header=True)

# Show DataFrame
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   27288431| R33UPQQUZQEM8|B005T4ND06|     400024643|Yoga for Movement...|       Video DVD|          5|            3|          3|   N|                Y|This was a gift f...|This was a gift f...| 2015-08-31|
|         US|   13722556|R3IKTNQQPD9662|B004EPZ070|     685335564|  Something Borrowed|       Video DVD|          5|    

In [9]:
vine_table = df.select('review_id','star_rating','helpful_votes','total_votes','vine')
vine_table.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| R33UPQQUZQEM8|          5|            3|          3|   N|
|R3IKTNQQPD9662|          5|            0|          0|   N|
|R3U27V5QMCP27T|          5|            1|          1|   N|
|R2TOH2QKNK4IOC|          5|            0|          1|   N|
|R2XQG5NJ59UFMY|          5|            0|          0|   N|
|R1N1KHBRR4ZTX3|          5|            0|          0|   N|
|R3OM9S0TCBP38K|          5|            0|          0|   N|
|R1W4S949ZRCTBW|          5|            0|          0|   N|
|R18JL1NNQAZFV2|          5|            0|          0|   N|
|R1LP6PR06OPYUX|          4|            0|          0|   N|
| RZKBT035JA0UQ|          5|            1|          2|   N|
|R253N5W74SM7N3|          4|            1|          1|   N|
|R2D5IFTFPHD3RN|          4|            1|          1|   N|
|R1CECK3H1URK1G|          5|            

In [10]:
review_id_table = df.select("review_id", "customer_id",'product_id','product_parent','review_date')
review_id_table.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| R33UPQQUZQEM8|   27288431|B005T4ND06|     400024643| 2015-08-31|
|R3IKTNQQPD9662|   13722556|B004EPZ070|     685335564| 2015-08-31|
|R3U27V5QMCP27T|   20381037|B005S9EKCW|     922008804| 2015-08-31|
|R2TOH2QKNK4IOC|   24852644|B00FC1ZCB4|     326560548| 2015-08-31|
|R2XQG5NJ59UFMY|   15556113|B002ZG98Z0|     637495038| 2015-08-31|
|R1N1KHBRR4ZTX3|    6132474|B00X8RONBO|     896602391| 2015-08-31|
|R3OM9S0TCBP38K|   48049524|B000CEXFZG|     115883890| 2015-08-31|
|R1W4S949ZRCTBW|    3282516|B00ID8H8EW|     977932459| 2015-08-31|
|R18JL1NNQAZFV2|   51771179|B000TGJ8IU|     840084782| 2015-08-31|
|R1LP6PR06OPYUX|   31816501|B00DPMPTDS|     262144920| 2015-08-31|
| RZKBT035JA0UQ|   16164990|B00X797LUS|     883589001| 2015-08-31|
|R253N5W74SM7N3|   33386989|B00C6MXB42|     734735137| 2015-08

In [11]:
products = df.select("product_id", "product_title")
products.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B005T4ND06|Yoga for Movement...|
|B004EPZ070|  Something Borrowed|
|B005S9EKCW|Les Miserables (2...|
|B00FC1ZCB4|Alien Anthology a...|
|B002ZG98Z0|  Sex and the City 2|
|B00X8RONBO|When Calls The He...|
|B000CEXFZG|Teen Titans - The...|
|B00ID8H8EW|      Generation War|
|B000TGJ8IU|Troy  (Director's...|
|B00DPMPTDS|Faith Aka the Gre...|
|B00X797LUS|   Revenge: Season 4|
|B00C6MXB42|YOUNG INDIANA JON...|
|B000EZ9084|     Survival Island|
|B000CEXFZG|Teen Titans - The...|
|B00KXEM6XM|     Fargo: Season 1|
|B0036K9CM8|Disney Parks: The...|
|B00G3HOJZ6|Young Detective D...|
|B00V5E7YR2|The Divergent Ser...|
|B005BUA1JY|Barney Miller: Th...|
|B00YCY46VO|Inside Out (Blu-r...|
+----------+--------------------+
only showing top 20 rows



In [12]:
customers = df.select("customer_id")
customers.show()

+-----------+
|customer_id|
+-----------+
|   27288431|
|   13722556|
|   20381037|
|   24852644|
|   15556113|
|    6132474|
|   48049524|
|    3282516|
|   51771179|
|   31816501|
|   16164990|
|   33386989|
|   45486371|
|   14006420|
|   23411619|
|    3202332|
|   32012808|
|   36507765|
|   13696097|
|    2190805|
+-----------+
only showing top 20 rows



In [13]:
customers = df.groupBy('customer_id').count().withColumnRenamed('count','customer_count')
customers.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|    9507671|            13|
|   15080145|             1|
|   25072529|             1|
|   31325270|             2|
|    9058434|             2|
|     220616|             3|
|   48428870|            25|
|   22848554|             8|
|    7763633|             1|
|   43482708|            12|
|   24139636|             1|
|    5884899|             7|
|   42200307|             1|
|   43963799|             1|
|   36879575|             1|
|   17345393|             6|
|   18460373|             3|
|   29351344|             1|
|   52809621|             2|
|   52857540|             1|
+-----------+--------------+
only showing top 20 rows



In [14]:
# Configuration for RDS instance
mode="append"
jdbc_url = "jdbc:postgresql://127.0.0.1:5432/postgres"
config = {"user":"postgres",
          "password": "PostGresPasscode",
          "driver":'org.postgresql.Driver'}

In [15]:
customers.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config) 

Py4JJavaError: ignored